In [1]:
# Установка pyspark
!pip install pyspark==3.0.3

Defaulting to user installation because normal site-packages is not writeable


In [1]:
# Библиотеки
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, countDistinct
from pyspark.sql.types import StructType, StructField, IntegerType
from pyspark.sql.types import TimestampType, FloatType, StringType
import subprocess
import os

In [16]:
# Получаем список файлов в bucket
cmd = "hdfs dfs -ls s3a://cherepanov-bucket-new/"
files = subprocess.check_output(cmd, shell=True).decode("utf-8").split("\n")
files = [f.split(" ")[-1] for f in files if ".txt" in f]

In [17]:
# Создаем сессию spark
spark = SparkSession.builder.config("spark.hadoop.fs.s3a.access.key", "YCAJEu8SE1sWgkwmzquSB8ktL") \
                              .config("spark.hadoop.fs.s3a.secret.key", "YCM7SiqO1z6BNvoU4mgEqZyg_PphP0IQnncQ8lkz") \
                              .config("spark.hadoop.fs.endpoint", "storage.yandexcloud.net") \
                              .getOrCreate()

In [18]:
# Создаем схему данных
schema = StructType(
    [
        StructField("tranaction_id", IntegerType(), True),
        StructField("tx_datetime", TimestampType(), True),
        StructField("customer_id", IntegerType(), True),
        StructField("terminal_id", IntegerType(), True),
        StructField("tx_amount", FloatType(), True),
        StructField("tx_time_seconds", IntegerType(), True),
        StructField("tx_time_days", IntegerType(), True),
        StructField("tx_fraud", IntegerType(), True),
        StructField("tx_fraud_scenario", IntegerType(), True),
    ]
)

In [21]:
# Прочитаем первый csv для его анализа
df = spark.read.format("csv") \
                 .schema(schema) \
                 .option("header", "true") \
                 .option("sep", ",") \
                 .load(files[0])
df.show(n=2)

+-------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|tranaction_id|        tx_datetime|customer_id|terminal_id|tx_amount|tx_time_seconds|tx_time_days|tx_fraud|tx_fraud_scenario|
+-------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|            0|2019-08-22 06:51:03|          0|        711|    70.91|          24663|           0|       0|                0|
|            1|2019-08-22 05:10:37|          0|          0|    90.55|          18637|           0|       0|                0|
+-------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
only showing top 2 rows



In [20]:
df.printSchema()

root
 |-- tranaction_id: integer (nullable = true)
 |-- tx_datetime: timestamp (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- terminal_id: integer (nullable = true)
 |-- tx_amount: float (nullable = true)
 |-- tx_time_seconds: integer (nullable = true)
 |-- tx_time_days: integer (nullable = true)
 |-- tx_fraud: integer (nullable = true)
 |-- tx_fraud_scenario: integer (nullable = true)



In [22]:
# список имен колонок
cols = list(df.columns)

In [23]:
# считаем NaN
columns_has_null = [(col, df.filter(df[col].isNull()).count()) for col in cols]
columns_has_null

[('tranaction_id', 0),
 ('tx_datetime', 100),
 ('customer_id', 0),
 ('terminal_id', 0),
 ('tx_amount', 0),
 ('tx_time_seconds', 0),
 ('tx_time_days', 0),
 ('tx_fraud', 0),
 ('tx_fraud_scenario', 0)]

In [24]:
# удаляем строки с пропусками
df2 = df.na.drop()
df2.count()

46988318

In [25]:
# считаем дубли
counts_df = df2.select(
    [
        countDistinct(*cols).alias('n_unique'),
        count('*').alias('n_rows')
    ]
)
n_unique, n_rows = counts_df.collect()[0]
print(n_unique, n_rows)

46988137 46988318


In [26]:
# удаляем дубли
df3 = df2.drop_duplicates()
df3.count()

46988137

In [27]:
# Останавливаем работу spark
spark.stop()